In [3]:
import time
import pandas as pd
import numpy as np
import openml
import warnings
import math
import logging

# Imports do scikit-learn e auto-sklearn
from autosklearn.experimental.askl2 import AutoSklearn2Classifier

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score, accuracy_score, log_loss

In [4]:
# Configurações
SEED = 42
warnings.filterwarnings("ignore", category=FutureWarning)

def carregar_base_openml(openml_id):
    """Carrega um dataset do OpenML e prepara os tipos de dados."""
    dataset = openml.datasets.get_dataset(openml_id, download_data=True, download_qualities=True, download_features_meta_data=True)
    X, y, _, _ = dataset.get_data(dataset_format="dataframe", target=dataset.default_target_attribute)
    if isinstance(y, pd.Series):
        y = y.values.ravel()
    return X, y

In [8]:
def run_askl_experiment(openml_id):
    """
    Executa o fluxo completo de avaliação para o auto-sklearn em um dataset.
    """
    print(f"--- Processando Dataset ID: {openml_id} com auto-sklearn ---")
    
    # Orçamento de tempo em segundos. Use 3600 para os resultados finais.
    AUTOML_TIME_BUDGET = 3600  # 10 minutos para um teste rápido

    # 1. Carregamento e pré-processamento
    X_df, y_orig = carregar_base_openml(openml_id)
    
    X_train_df, X_test_df, y_train_orig, y_test_orig = train_test_split(
        X_df, y_orig, test_size=0.3, random_state=SEED, stratify=(y_orig if len(np.unique(y_orig)) > 1 else None)
    )

    le = LabelEncoder()
    y_train_enc = le.fit_transform(y_train_orig)
    y_test_enc = le.transform(y_test_orig)

    # Imputação de NaNs numéricos
    num_cols = X_train_df.select_dtypes(include=np.number).columns
    if len(num_cols) > 0 and X_train_df[num_cols].isnull().sum().sum() > 0:
        imp = SimpleImputer(strategy='median')
        X_train_df.loc[:, num_cols] = imp.fit_transform(X_train_df[num_cols])
        X_test_df.loc[:, num_cols] = imp.transform(X_test_df[num_cols])

    # Imputação de NaNs categóricos (convertendo para string)
    cat_cols = X_train_df.select_dtypes(include=['category', 'object']).columns
    if len(cat_cols) > 0:
        for col in cat_cols:
            X_train_df.loc[:, col] = X_train_df[col].astype(str).fillna("__MISSING__")
            X_test_df.loc[:, col] = X_test_df[col].astype(str).fillna("__MISSING__")
            
    # 2. Treinamento com auto-sklearn
    t0_total = time.time()

    automl = AutoSklearn2Classifier(
        time_left_for_this_task=AUTOML_TIME_BUDGET,
        n_jobs=-1,                
        seed=SEED,
        memory_limit=None
    )
    
    # O auto-sklearn lida bem com DataFrames pré-processados
    automl.fit(X_train_df, y_train_enc)
    
    # 3. Predição e Métricas
    y_proba = automl.predict_proba(X_test_df)
    y_pred = automl.predict(X_test_df)
    total_time = time.time() - t0_total

    n_classes = len(le.classes_)
    if n_classes == 2:
        auc_ovo = roc_auc_score(y_test_enc, y_proba[:, 1])
    else:
        auc_ovo = roc_auc_score(y_test_enc, y_proba, multi_class='ovo')

    acc = accuracy_score(y_test_enc, y_pred)
    ce = log_loss(y_test_enc, y_proba, labels=range(n_classes))
    
    # Extrai o melhor score de validação cruzada dos resultados internos
    best_score_cv = np.max(automl.cv_results_['mean_test_score'])


    print(f"Resultados para {openml_id}: AUC={auc_ovo:.4f}, ACC={acc:.4f}, Time={total_time:.2f}s")
    
    return {
        'openml_id': openml_id, 'modelo': 'autosklearn', 'total_time_sec': total_time,
        'mean_auc_ovo': auc_ovo, 'mean_accuracy': acc, 'mean_cross_entropy': ce,
        'best_model_info': 'Ensemble', # O melhor modelo é um ensemble complexo
        'best_score_cv': best_score_cv
    }

In [ ]:
logging.getLogger("autosklearn").setLevel(logging.INFO)
MACHINE_ID = 4
TOTAL_MACHINES = 4

cc18_ids = [
    11, 15, 18, 23, 29, 31, 37, 50, 54, 188,
    307, 458, 469, 1049, 1050, 1063, 1068, 1462, 1464, 1468,
    1480, 1494, 1501, 1510, 6332, 23381, 40966, 40975, 40982, 40994
]

chunk_size = math.ceil(len(cc18_ids) / TOTAL_MACHINES)
start_index = (MACHINE_ID - 1) * chunk_size
end_index = start_index + chunk_size
ids_to_process = cc18_ids[start_index:end_index]

print(f"--- MÁQUINA {MACHINE_ID}/{TOTAL_MACHINES} ---")
print(f"Processando os seguintes datasets: {ids_to_process}")
    
todos_resultados_askl = []

for oid in ids_to_process:
    try:
        res = run_askl_experiment(oid)
        todos_resultados_askl.append(res)
    except Exception as e:
        print(f"!!!!!! ERRO AO PROCESSAR O DATASET {oid}: {e} !!!!!!")
        import traceback
        traceback.print_exc()
        continue

if todos_resultados_askl:
    df_askl_results = pd.DataFrame(todos_resultados_askl)
    print("\n\n--- RESULTADOS FINAIS DO AUTO-SKLEARN ---")
    print(df_askl_results)
    df_askl_results.to_csv("resultados_autosklearn.csv", index=False)
    print("\nArquivo 'resultados_autosklearn.csv' salvo com sucesso!")
else:
    print("\nNenhuma tarefa foi concluída com sucesso.")

--- MÁQUINA 4/4 ---
Processando os seguintes datasets: [6332, 23381, 40966, 40975, 40982, 40994]
--- Processando Dataset ID: 6332 com auto-sklearn ---


/var/tmp/ipykernel_29820/1891648276.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_df.loc[:, num_cols] = imp.fit_transform(X_train_df[num_cols])
/var/tmp/ipykernel_29820/1891648276.py:25: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X_train_df.loc[:, num_cols] = imp.fit_transform(X_train_df[num_cols])
/var/tmp/ipykernel_29820/1891648276.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

[WARNING] [2025-06-15 01:04:36,994:Client-AutoML(42):abbe15ed-4984-11f0-b47c-42010a9e0009] Capping the per_run_time_limit to 1799.0 to have time for a least 2 models in each process.
[ERROR] [2025-06-15 01:04:37,602:Client-AutoML(42):abbe15ed-4984-11f0-b47c-42010a9e0009] list index out of range
Traceback (most recent call last):
  File "/opt/conda/envs/askl_env/lib/python3.9/site-packages/autosklearn/automl.py", line 899, in fit
    ) = _proc_smac.run_smbo()
  File "/opt/conda/envs/askl_env/lib/python3.9/site-packages/autosklearn/smbo.py", line 552, in run_smbo
    smac.optimize()
  File "/opt/conda/envs/askl_env/lib/python3.9/site-packages/smac/facade/smac_ac_facade.py", line 720, in optimize
    incumbent = self.solver.run()
  File "/opt/conda/envs/askl_env/lib/python3.9/site-packages/smac/optimizer/smbo.py", line 211, in run
    self.start()
  File "/opt/conda/envs/askl_env/lib/python3.9/site-packages/smac/optimizer/smbo.py", line 174, in start
    self.initial_design_configs = self

Traceback (most recent call last):
  File "/var/tmp/ipykernel_29820/3753875776.py", line 23, in <module>
    res = run_askl_experiment(oid)
  File "/var/tmp/ipykernel_29820/1891648276.py", line 46, in run_askl_experiment
    automl.fit(X_train_df, y_train_enc)
  File "/opt/conda/envs/askl_env/lib/python3.9/site-packages/autosklearn/experimental/askl2.py", line 540, in fit
    return super().fit(
  File "/opt/conda/envs/askl_env/lib/python3.9/site-packages/autosklearn/estimators.py", line 1448, in fit
    super().fit(
  File "/opt/conda/envs/askl_env/lib/python3.9/site-packages/autosklearn/estimators.py", line 540, in fit
    self.automl_.fit(load_models=self.load_models, **kwargs)
  File "/opt/conda/envs/askl_env/lib/python3.9/site-packages/autosklearn/automl.py", line 2304, in fit
    return super().fit(
  File "/opt/conda/envs/askl_env/lib/python3.9/site-packages/autosklearn/automl.py", line 962, in fit
    raise e
  File "/opt/conda/envs/askl_env/lib/python3.9/site-packages/autoskle

[WARNING] [2025-06-15 01:04:50,988:Client-AutoML(42):b91d44dc-4984-11f0-b47c-42010a9e0009] Capping the per_run_time_limit to 1799.0 to have time for a least 2 models in each process.
[ERROR] [2025-06-15 01:04:51,104:Client-AutoML(42):b91d44dc-4984-11f0-b47c-42010a9e0009] list index out of range
Traceback (most recent call last):
  File "/opt/conda/envs/askl_env/lib/python3.9/site-packages/autosklearn/automl.py", line 899, in fit
    ) = _proc_smac.run_smbo()
  File "/opt/conda/envs/askl_env/lib/python3.9/site-packages/autosklearn/smbo.py", line 552, in run_smbo
    smac.optimize()
  File "/opt/conda/envs/askl_env/lib/python3.9/site-packages/smac/facade/smac_ac_facade.py", line 720, in optimize
    incumbent = self.solver.run()
  File "/opt/conda/envs/askl_env/lib/python3.9/site-packages/smac/optimizer/smbo.py", line 211, in run
    self.start()
  File "/opt/conda/envs/askl_env/lib/python3.9/site-packages/smac/optimizer/smbo.py", line 174, in start
    self.initial_design_configs = self

...

[WARNING] [2025-06-15 01:05:00,740:Client-AutoML(42):c153adf6-4984-11f0-b47c-42010a9e0009] Capping the per_run_time_limit to 1799.0 to have time for a least 2 models in each process.
Resultados para 40966: AUC=1.0000, ACC=1.0000, Time=3618.25s
--- Processando Dataset ID: 40975 com auto-sklearn ---


/var/tmp/ipykernel_29820/1891648276.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_df.loc[:, col] = X_train_df[col].astype(str).fillna("__MISSING__")
/var/tmp/ipykernel_29820/1891648276.py:32: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X_train_df.loc[:, col] = X_train_df[col].astype(str).fillna("__MISSING__")
/var/tmp/ipykernel_29820/1891648276.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

[WARNING] [2025-06-15 02:05:37,806:Client-AutoML(42):30944664-498d-11f0-b47c-42010a9e0009] Capping the per_run_time_limit to 1799.0 to have time for a least 2 models in each process.
[ERROR] [2025-06-15 02:05:37,940:Client-AutoML(42):30944664-498d-11f0-b47c-42010a9e0009] list index out of range
Traceback (most recent call last):
  File "/opt/conda/envs/askl_env/lib/python3.9/site-packages/autosklearn/automl.py", line 899, in fit
    ) = _proc_smac.run_smbo()
  File "/opt/conda/envs/askl_env/lib/python3.9/site-packages/autosklearn/smbo.py", line 552, in run_smbo
    smac.optimize()
  File "/opt/conda/envs/askl_env/lib/python3.9/site-packages/smac/facade/smac_ac_facade.py", line 720, in optimize
    incumbent = self.solver.run()
  File "/opt/conda/envs/askl_env/lib/python3.9/site-packages/smac/optimizer/smbo.py", line 211, in run
    self.start()
  File "/opt/conda/envs/askl_env/lib/python3.9/site-packages/smac/optimizer/smbo.py", line 174, in start
    self.initial_design_configs = self

Traceback (most recent call last):
  File "/var/tmp/ipykernel_29820/3753875776.py", line 23, in <module>
    res = run_askl_experiment(oid)
  File "/var/tmp/ipykernel_29820/1891648276.py", line 46, in run_askl_experiment
    automl.fit(X_train_df, y_train_enc)
  File "/opt/conda/envs/askl_env/lib/python3.9/site-packages/autosklearn/experimental/askl2.py", line 540, in fit
    return super().fit(
  File "/opt/conda/envs/askl_env/lib/python3.9/site-packages/autosklearn/estimators.py", line 1448, in fit
    super().fit(
  File "/opt/conda/envs/askl_env/lib/python3.9/site-packages/autosklearn/estimators.py", line 540, in fit
    self.automl_.fit(load_models=self.load_models, **kwargs)
  File "/opt/conda/envs/askl_env/lib/python3.9/site-packages/autosklearn/automl.py", line 2304, in fit
    return super().fit(
  File "/opt/conda/envs/askl_env/lib/python3.9/site-packages/autosklearn/automl.py", line 962, in fit
    raise e
  File "/opt/conda/envs/askl_env/lib/python3.9/site-packages/autoskle

[WARNING] [2025-06-15 02:05:53,016:Client-AutoML(42):418a29c6-498d-11f0-b47c-42010a9e0009] Capping the per_run_time_limit to 1799.0 to have time for a least 2 models in each process.
Resultados para 40982: AUC=0.9707, ACC=0.8199, Time=3620.58s
--- Processando Dataset ID: 40994 com auto-sklearn ---
[WARNING] [2025-06-15 03:06:21,472:Client-AutoML(42):b2a38e09-4995-11f0-b47c-42010a9e0009] Capping the per_run_time_limit to 1799.0 to have time for a least 2 models in each process.


In [5]:
from autosklearn.classification import AutoSklearnClassifier

def run_askl_experiment_optional(openml_id):
    """
    Executa o fluxo completo de avaliação para o auto-sklearn em um dataset.
    """
    print(f"--- Processando Dataset ID: {openml_id} com auto-sklearn ---")
    
    # Orçamento de tempo em segundos. Use 3600 para os resultados finais.
    AUTOML_TIME_BUDGET = 3600  # 10 minutos para um teste rápido

    # 1. Carregamento e pré-processamento
    X_df, y_orig = carregar_base_openml(openml_id)
    
    X_train_df, X_test_df, y_train_orig, y_test_orig = train_test_split(
        X_df, y_orig, test_size=0.3, random_state=SEED, stratify=(y_orig if len(np.unique(y_orig)) > 1 else None)
    )

    le = LabelEncoder()
    y_train_enc = le.fit_transform(y_train_orig)
    y_test_enc = le.transform(y_test_orig)

    # Imputação de NaNs numéricos
    num_cols = X_train_df.select_dtypes(include=np.number).columns
    if len(num_cols) > 0 and X_train_df[num_cols].isnull().sum().sum() > 0:
        imp = SimpleImputer(strategy='median')
        X_train_df.loc[:, num_cols] = imp.fit_transform(X_train_df[num_cols])
        X_test_df.loc[:, num_cols] = imp.transform(X_test_df[num_cols])

    # Imputação de NaNs categóricos (convertendo para string)
    cat_cols = X_train_df.select_dtypes(include=['category', 'object']).columns
    if len(cat_cols) > 0:
        for col in cat_cols:
            X_train_df.loc[:, col] = X_train_df[col].astype(str).fillna("__MISSING__")
            X_test_df.loc[:, col] = X_test_df[col].astype(str).fillna("__MISSING__")
            
    # 2. Treinamento com auto-sklearn
    t0_total = time.time()

    automl = AutoSklearnClassifier(
        time_left_for_this_task=AUTOML_TIME_BUDGET,
        n_jobs=-1,                
        seed=SEED,
        memory_limit=None
        #initial_configurations_via_metalearning=0
    )
    
    # O auto-sklearn lida bem com DataFrames pré-processados
    automl.fit(X_train_df, y_train_enc)
    
    # 3. Predição e Métricas
    y_proba = automl.predict_proba(X_test_df)
    y_pred = automl.predict(X_test_df)
    total_time = time.time() - t0_total

    n_classes = len(le.classes_)
    if n_classes == 2:
        auc_ovo = roc_auc_score(y_test_enc, y_proba[:, 1])
    else:
        auc_ovo = roc_auc_score(y_test_enc, y_proba, multi_class='ovo')

    acc = accuracy_score(y_test_enc, y_pred)
    ce = log_loss(y_test_enc, y_proba, labels=range(n_classes))
    
    # Extrai o melhor score de validação cruzada dos resultados internos
    best_score_cv = np.max(automl.cv_results_['mean_test_score'])


    print(f"Resultados para {openml_id}: AUC={auc_ovo:.4f}, ACC={acc:.4f}, Time={total_time:.2f}s")
    
    return {
        'openml_id': openml_id, 'modelo': 'autosklearn', 'total_time_sec': total_time,
        'mean_auc_ovo': auc_ovo, 'mean_accuracy': acc, 'mean_cross_entropy': ce,
        'best_model_info': 'Ensemble', # O melhor modelo é um ensemble complexo
        'best_score_cv': best_score_cv
    }

In [6]:
ids_to_process =  [6332, 23381, 40975]

print(f"Processando os seguintes datasets: {ids_to_process}")
    
todos_resultados_askl = []

for oid in ids_to_process:
    try:
        res = run_askl_experiment_optional(oid)
        todos_resultados_askl.append(res)
    except Exception as e:
        print(f"!!!!!! ERRO AO PROCESSAR O DATASET {oid}: {e} !!!!!!")
        import traceback
        traceback.print_exc()
        continue

if todos_resultados_askl:
    df_askl_results = pd.DataFrame(todos_resultados_askl)
    print("\n\n--- RESULTADOS FINAIS DO AUTO-SKLEARN ---")
    print(df_askl_results)
    df_askl_results.to_csv("resultados_autosklearn2.csv", index=False)
    print("\nArquivo 'resultados_autosklearn.csv' salvo com sucesso!")
else:
    print("\nNenhuma tarefa foi concluída com sucesso.")

Processando os seguintes datasets: [6332, 23381, 40975]
--- Processando Dataset ID: 6332 com auto-sklearn ---


/var/tmp/ipykernel_3326/2533747859.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_df.loc[:, num_cols] = imp.fit_transform(X_train_df[num_cols])
/var/tmp/ipykernel_3326/2533747859.py:27: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X_train_df.loc[:, num_cols] = imp.fit_transform(X_train_df[num_cols])
/var/tmp/ipykernel_3326/2533747859.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

[WARNING] [2025-06-15 07:56:48,836:Client-AutoML(42):493aecba-49be-11f0-8cfe-42010a9e0009] Capping the per_run_time_limit to 1799.0 to have time for a least 2 models in each process.
[WARNING] [2025-06-15 07:56:52,467:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2025-06-15 07:56:52,662:Client-EnsembleBuilder] No runs were available to build an ensemble from
Resultados para 6332: AUC=0.8265, ACC=0.7469, Time=3608.53s
--- Processando Dataset ID: 23381 com auto-sklearn ---


/var/tmp/ipykernel_3326/2533747859.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_df.loc[:, col] = X_train_df[col].astype(str).fillna("__MISSING__")
/var/tmp/ipykernel_3326/2533747859.py:34: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X_train_df.loc[:, col] = X_train_df[col].astype(str).fillna("__MISSING__")
/var/tmp/ipykernel_3326/2533747859.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

[WARNING] [2025-06-15 08:56:57,601:Client-AutoML(42):b0307dda-49c6-11f0-8cfe-42010a9e0009] Capping the per_run_time_limit to 1799.0 to have time for a least 2 models in each process.
[WARNING] [2025-06-15 08:57:00,597:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2025-06-15 08:57:01,923:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2025-06-15 08:57:02,906:Client-EnsembleBuilder] No models better than random - using Dummy losses!
	Models besides current dummy model: 0
	Dummy models: 1
Resultados para 23381: AUC=0.5249, ACC=0.5733, Time=3599.61s
--- Processando Dataset ID: 40975 com auto-sklearn ---


/var/tmp/ipykernel_3326/2533747859.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_df.loc[:, col] = X_train_df[col].astype(str).fillna("__MISSING__")
/var/tmp/ipykernel_3326/2533747859.py:34: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X_train_df.loc[:, col] = X_train_df[col].astype(str).fillna("__MISSING__")
/var/tmp/ipykernel_3326/2533747859.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

[WARNING] [2025-06-15 09:57:13,053:Client-AutoML(42):11cbb82c-49cf-11f0-8cfe-42010a9e0009] Capping the per_run_time_limit to 1799.0 to have time for a least 2 models in each process.
Resultados para 40975: AUC=0.9638, ACC=0.7900, Time=3616.01s


--- RESULTADOS FINAIS DO AUTO-SKLEARN ---
   openml_id       modelo  total_time_sec  mean_auc_ovo  mean_accuracy  \
0       6332  autosklearn     3608.533348      0.826502       0.746914   
1      23381  autosklearn     3599.608409      0.524904       0.573333   
2      40975  autosklearn     3616.009416      0.963769       0.789981   

   mean_cross_entropy best_model_info  best_score_cv  
0            0.503407        Ensemble       0.776000  
1            0.688782        Ensemble       0.706897  
2            0.602161        Ensemble       0.834586  

Arquivo 'resultados_autosklearn.csv' salvo com sucesso!
